# LLM Named entity recognition 

In [10]:
!python -m spacy download pl_core_news_sm

Traceback (most recent call last):
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 148, in _get_module_details
  File "<frozen runpy>", line 112, in _get_module_details
  File "c:\Users\janek\Documents\PJN\venv\Lib\site-packages\spacy\__init__.py", line 6, in <module>
    from .errors import setup_default_warnings
  File "c:\Users\janek\Documents\PJN\venv\Lib\site-packages\spacy\errors.py", line 3, in <module>
    from .compat import Literal
  File "c:\Users\janek\Documents\PJN\venv\Lib\site-packages\spacy\compat.py", line 39, in <module>
    from thinc.api import Optimizer  # noqa: F401
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\janek\Documents\PJN\venv\Lib\site-packages\thinc\api.py", line 1, in <module>
    from .backends import (
  File "c:\Users\janek\Documents\PJN\venv\Lib\site-packages\thinc\backends\__init__.py", line 17, in <module>
    from .cupy_ops import CupyOps
  File "c:\Users\janek\Documents\PJN\venv\Lib\site-packages\th

### Loading 20 random passages from FIQA-PL Corpus

In [1]:
from datasets import load_dataset

corpus = load_dataset("clarin-knext/fiqa-pl", "corpus")
corpus = corpus['corpus']
sampled_corpus = corpus.shuffle(seed=42).select([i for i in range(20)])
sampled_corpus

c:\Users\janek\Documents\PJN\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['_id', 'title', 'text'],
    num_rows: 20
})

### Creating spacy pipeline for NER

In [2]:
from spacy.lang.pl import Polish
import spacy
nlp = Polish()
nlp = spacy.load("pl_core_news_sm") # includes lemmatizer and ner
nlp.replace_pipe("lemmatizer", "lemmatizer")
nlp.add_pipe("sentencizer")
nlp.initialize()
nlp.components

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x1e419801850>),
 ('morphologizer',
  <spacy.pipeline.morphologizer.Morphologizer at 0x1e4198011f0>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x1e4197c1ee0>),
 ('lemmatizer', <spacy.lang.pl.lemmatizer.PolishLemmatizer at 0x1e4196c13d0>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x1e4197abf50>),
 ('senter', <spacy.pipeline.senter.SentenceRecognizer at 0x1e4197abad0>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x1e41983aa90>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x1e4197c1f50>),
 ('sentencizer', <spacy.pipeline.sentencizer.Sentencizer at 0x1e419845810>)]

In [13]:
# def get_entities_with_ner(text):
#   entities = []
#   doc = nlp(text)
#   for i in doc.ents:
#       entities.append({"ent": i.lemma_, "type": i.label_})
#   return entities

# print(get_entities_with_ner("Wczoraj w Krakowie miało miejsce spotkanie prezydentów Polski i Stanów Zjednoczonych."))

def get_entities_no_ner(text):
  entities = set()
  doc = nlp(text, disable=["ner"])
  processed = nlp(doc)

  # Variable to hold the current entity being built
  entity = ""

  for i in processed:
      # Check if the lemma of the token starts with an uppercase letter
      if i.lemma_[0].isupper():
          # Start a new entity if needed or continue appending to the current one
          if not entity:
              entity = i.lemma_ 
          else:
              entity += " " + i.lemma_
      else:
          # Print the current entity if it exists and reset
          if entity:
              entities.add(entity)
              entity = ""  # Reset for the next potential entity

  return entities

print(get_entities_no_ner("Wczoraj w Krakowie miało miejsce spotkanie prezydentów Polski i Stanów Zjednoczonych."))

{'Kraków', 'Polska'}


c:\Users\janek\Documents\PJN\venv\Lib\site-packages\spacy\pipeline\attributeruler.py:149: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  matches = self.matcher(doc, allow_missing=True, as_spans=False)


### Manually annotating 20 passages from Corpus

In [8]:
for doc in sampled_corpus:
    print(doc["text"])
    print()

Według Nanex, firmy zajmującej się danymi rynkowymi, program składał zamówienia w 25-milisekundowych seriach obejmujących około 500 akcji. Algorytm nigdy nie wykonał ani jednej transakcji i nagle zakończył się około godziny 10:30 w piątek ET. Więc za każdym razem zmieniał zdanie? To albo błąd, albo działa z przodu. Myślę, że to frontowe bieganie bez względu na to, jak na to patrzysz. Gdybym uruchomił SEC, wprowadziłbym regułę, która mówi, że wszystkie zamówienia muszą stać przez 2 sekundy, zanim będą mogły zostać anulowane. To wystarczająco dużo czasu, aby ludzie zareagowali na rynku. To 25 ms dla 500 akcji to nonsens. To tylko front, który ma oszukiwać prawdziwych inwestorów i zarabiać na bardzo małych różnicach w cenach miliony razy dziennie. Zniekształca rynki i nie służy nikomu, z wyjątkiem domu maklerskiego, który prowadzi oszustwo.

ETF-y kochanie. Wszyscy i ich mama mają swoje pieniądze na rynkach / w awangardzie Gdy na rynkach nastąpi nieunikniona korekta lub jedna z firm techn

Według Nanex, firmy zajmującej się danymi rynkowymi, program składał zamówienia w 25-milisekundowych seriach obejmujących około 500 akcji. Algorytm nigdy nie wykonał ani jednej transakcji i nagle zakończył się około godziny 10:30 w piątek ET. Więc za każdym razem zmieniał zdanie? To albo błąd, albo działa z przodu. Myślę, że to frontowe bieganie bez względu na to, jak na to patrzysz. Gdybym uruchomił SEC, wprowadziłbym regułę, która mówi, że wszystkie zamówienia muszą stać przez 2 sekundy, zanim będą mogły zostać anulowane. To wystarczająco dużo czasu, aby ludzie zareagowali na rynku. To 25 ms dla 500 akcji to nonsens. To tylko front, który ma oszukiwać prawdziwych inwestorów i zarabiać na bardzo małych różnicach w cenach miliony razy dziennie. Zniekształca rynki i nie służy nikomu, z wyjątkiem domu maklerskiego, który prowadzi oszustwo.

ETF-y kochanie. Wszyscy i ich mama mają swoje pieniądze na rynkach / w awangardzie Gdy na rynkach nastąpi nieunikniona korekta lub jedna z firm technologicznych zorganizuje wydarzenie korporacyjne, które będzie obciążać branżę: zaczną krwawić przepływy. A to zmniejszy zwroty. Co doprowadzi do większej liczby przepływów. Co zmniejszy zwroty. Itd.. i itd. W ten sposób łamią ramy rynkowe. Firmy są silnie lewarowane ze względu na niskie stopy procentowe, więc gdy wskaźniki zadłużenia do kapitału zaczynają rosnąć – nabywcy indywidualni/instytucjonalni zaczynają uciekać w stronę jakości i zaczynają wycofywać się z małych, średnich kapitalizacji z dźwignią. Pęknięcia w małych firmach. Trump ratuje sytuację dzięki podatkom korporacyjnym! I wszyscy płoniemy w północnokoreańskim ogniu

Pozwólmy wolnemu rynkowi działać tak, aby najlepszy, najbardziej wydajny produkt, który jest najlepszy dla konsumentów, prosperował. Co za **niesamowity** pomysł. Ale niestety, to jest Reddit, gdzie narzekają na SOPA/PIPA i NDAA, ale wierzą w głosowanie w prawodawcach, którzy chcą rozszerzyć rolę rządu w prywatnym przedsiębiorstwie/życiu.

Nie. Otrzymałeś już odliczenie, ponieważ utracone pieniądze FSA były przed opodatkowaniem. Nie możesz się podwoić. Jeśli jednak kwota FSA była niższa niż rzeczywiste wydatki, możesz złożyć wniosek o różnicę. Przykład: 4500 USD rzeczywistych wydatków, 3000 USD FSA (utracone). Możesz otrzymać kredyt na opiekę na utrzymaniu w oparciu o 1500 USD.

Jeśli naprawdę jesteś gotów spłacać dodatkowe 1000 USD co miesiąc i masz pewność, że prawdopodobnie zawsze będziesz w stanie, powinieneś refinansować 15-letni kredyt hipoteczny. 15-letnie kredyty hipoteczne są zazwyczaj sprzedawane przy oprocentowaniu niższym o około pół punktu, co oznacza, że ​​zamiast 4,375% APR, otrzymasz około 3,875% APR. To dużo pieniędzy w trakcie kredytu hipotecznego. Zapłacisz około tysiąca więcej miesięcznie – a więc dokładnie to, o czym myślisz – i nie tylko zaoszczędzisz pieniądze z tej wcześniejszej płatności, ale także będziesz miał niższe oprocentowanie. To 0,5% oznacza mniej więcej 25 tys. USD w okresie spłaty kredytu hipotecznego. Jest to również różnica około 130 USD miesięcznie w wymaganej płatności. Teraz oczywiście będziesz zmuszony do dokonania tej większej płatności – więc różnica między tym, co sugerujesz, a tym, że płacisz dodatkowe 25 000 USD w zamian za możliwość wolniejszej spłaty (w takim przypadku oczywiście zapłaciłbyś również wyższe odsetki, ale w najlepszym przypadku). W scenariuszu 15 lat musisz dokonać tych ~4000 płatności. W scenariuszu 30 lat możesz zapłacić ~ 2900 USD za chwilę, jeśli stracisz pracę, chcesz wyjechać na wakacje lub ... cokolwiek. Oczywiście jest też na odwrót: będziesz musiał dokonać płatności, więc będziesz. Wiele osób uważa wymuszone oszczędności za dobrą strategię (ja wśród nich ja); Mam 15-letni kredyt hipoteczny i cieszę się, że muszę płacić wyższą ratę, bo to oznacza, że ​​nie mogę lekkomyślnie wydawać tych dodatkowych pieniędzy. Więc co bym zrobił na twoim miejscu, to rozejrzeć się za 15-letnim refi. To będzie kosztować kilka tysięcy, więc nie bierz jednego, chyba że możesz zaoszczędzić co najmniej pół punktu, ale jeśli możesz, zrób.

To nie miasto, to Zarząd Portu. To specjalna wielostanowa policja. PA jest powodem, dla którego most tapanzee został zbudowany w tak głupim miejscu. Most musiał zostać zbudowany 25 mil od Statuy Wolności, aby stan ny mógł pobierać opłaty. Jeśli został zbudowany w promieniu 25 mil, rządzi PA.

„Podstawowym równaniem nauczanym w pierwszym dniu szkoły rachunkowości jest to, że aktywa = pasywa + kapitał własny. Po pierwsze, zwróciłem uwagę na rzeczywiste sprawozdania finansowe opublikowane na koniec II kwartału 2017 r. i sumę zobowiązań na ich zbadanym bilansie arkusz był jak 13 miliardów dolarów, a nie 20 miliardów dolarów. Nie wiem, skąd autor wziął ich numery. Moja druga uwaga: dług zwykle musi być spłacany na wcześniej ustalonych warunkach uzgodnionych przez dłużnika i dłużnika, w tym odsetki, więc ważne jest, aby firma śledziła, co jest winna i komu, aby mogła dokonywać płatności na czas. O ile mają pod ręką gotówkę na dokonanie płatności oraz wszelkie odsetki, które są winni, a właściciele są zadowoleni z całkowitego zwrotu na ich inwestycję, to tak naprawdę nie ma znaczenia, jaki mają zadłużenie w bilansie. Zapamiętaj równanie A = L + E. Istnieją dokładnie dwa sposoby finansowania firmy, która chce nabyć aktywa: pasywa i/lub kapitał własny. „Właściwy” poziom zadłużenia vs kapitał własny w bilansie różni się znacznie i całkowicie zależy od branży, wielkości firmy, przepływów pieniężnych, osobistych preferencji dyrektora generalnego, dyrektora finansowego, akcjonariuszy itp. Oczywiście jest to o wiele bardziej szczegółowe i skomplikowane, ale chodzi o to, że patrzenie na sam dług jest bezsensowną miarą. To jest finanse i księgowość przedsiębiorstw 101, więc prawdopodobnie znajdziesz mnóstwo świetnych artykułów i filmów, jeśli chcesz dowiedzieć się więcej”.

To prawie miłe, ponieważ nie zwraca uwagi organizacji jako spamera. Jeśli organizacja zorientuje się, w jaki sposób znalazła się na liście, rozwiąże problem i wróci do dobrej pozycji, wtedy wszyscy będziemy mogli działać uprzejmie. Może istnieć nawet prawny powód, dla którego lista nie jest publiczna. Oszczerstwo?

„Wnioskodawcy przechodzą przez ten sam proces, co w przypadku każdej innej pożyczki. Jedyną różnicą jest to, że „bank” to ludzie tacy jak ja, być może 200 z nas na banknot. możliwość zadawania pytań wnioskodawcy Inwestorzy płacą niewielki procent opłat za wycenę banknotów przez fachowców Im gorszy kredyt, tym wyższe oprocentowanie otrzymuje posiadacz banknotu Oczywiście jeśli osoba przestaje płacić, nic nie otrzymujemy. ryzyko. Czasami inwestuję w banknoty o wysokim ryzyku, w zależności od tego, do czego są przeznaczone i mam ich kilka w swoim portfelu. To dosłownie mikropożyczanie. Mogę sobie pozwolić na stratę 25 USD na ryzykownej pożyczce lub dwóch na 19% zysku, ponieważ reszta mojego portfela jest zdywersyfikowana na setki nut 9-11%. Działa zaskakująco dobrze."

Zrób trochę matematyki. Oblicz sumę netto (po opodatkowaniu), którą miałbyś, gdybyś pozostawił ją opodatkowaną, a następnie spójrz na różnicę. W efekcie zapłacisz X USD (teraz podatek) za świadczenie Y USD (podatek od niezapłaconych pełnych świadczeń). Matematyka działa podobnie do samodzielnego kupowania małej polisy.

Człowieku, to zdumiewające, że rozwiązaniem bankiera jest zmniejszenie siły roboczej z 16 000 do 2000. Jasne, że będzie to dobrze wyglądać w bilansie, ale byłby to gwóźdź śmierci dla i tak już walczącej firmy. A ludzie zastanawiają się, dlaczego wszyscy są tak znudzeni tymi idiotami na Wall Street.

[Jest tak dobrze znany, że ACA będzie zawierał koszty w stosunku do niczego nie robienia, że ​​prawie nie warto cytować źródła.](http://www.nejm.org/doi/full/10.1056/NEJMp1006571) Zredagowano: To się udało złośliwy. Nie chcę ignorować znaczenia długoterminowego ograniczania kosztów w wydatkach na opiekę zdrowotną. Ale zamiast podnosić koszty, ACA jest tak naprawdę tylko małym, ale ważnym pierwszym krokiem w ograniczaniu kosztów. Więcej źródeł: [Affordable Care Act: Implementing Medicare Cost Savings](http://www.cms.gov/apps/docs/ACA-Update-Implementing-Medicare-Costs-Savings.pdf) [Affordable Care Act Will Bring Down Koszty](http://www.healthcare.gov/blog/2011/08/bringingdowncosts082911.html) [Jedynie pierwszy krok w ograniczaniu kosztów opieki zdrowotnej](http://www.nytimes.com/2012/07/22/ biznes/opieka-zdrowotna-prawo-i-ograniczanie-kosztow-gospodarczych-widok.html)

Dziadek kręci się, jesteś takim głupim pedałem. Zawsze mam nadzieję, że ktoś jest w liceum. >Warta tylko kpiny, będziesz się z ciebie śmiać przez następne 2 sekundy. miło widzieć, że wciąż myślisz, że to jest zabawne, urocze, boli, czy cokolwiek, kurwa, myślisz, że to jest.

Zazwyczaj pożyczasz firmie pożyczkę od siebie jako osoba prywatna, a gdy firma zarabia pieniądze, firma spłaca je Tobie. Następnie firma płaci za wszystkie wydatki pieniędzmi z kredytu. Nawet jeśli nie chcesz jeszcze konta firmowego, prawdopodobnie możesz poprosić swój bank o drugie konto (moje w Wielkiej Brytanii zrobiły to bez żadnych problemów).

Tak, stopnie MFE mogą być mieszaną torbą. Ogólnie rzecz biorąc, te programy się poprawiają i jeśli możesz dostać się do programu takiego jak NYU, UCB lub Carnegie Melon, to zdecydowanie polecam. Twoja praktyka z programu MFE będzie oparta na dwóch rzeczach: 1) Kogo zna dyrektor programu na Wall St. 2) Czy jesteś obcokrajowcem. Jeśli chcesz dostać pracę w Ameryce jako absolwent MFE, stoisz w obliczu trudnej bitwy w porównaniu z twoimi amerykańskimi odpowiednikami (chyba że idziesz do NYU, UCB, Carnegie Melon, a może MIT lub Chicago). Jeśli zdecydujesz się na doktorat, będzie to: fizyka, matematyka, informatyka lub ekonomia. Co więcej, przez cały okres swojej kariery będziesz ograniczony do roli badawczej. Większość z nich kończy przeprowadzanie eksperymentalnej/nowoczesnej analizy ryzyka i wyceny instrumentów pochodnych dla średniego biura. Od tej reguły zawsze są wyjątki; a ludzie z CS będą w dużej mierze inżynierami systemów transakcyjnych zaplecza. Jeśli interesuje Cię bycie traderem (najbardziej lukratywne stanowisko na Wall St. poza kadrą kierowniczą/menedżerami) lub menedżerem inwestycyjnym po stronie kupującego, nie musisz wychodzić poza MSF/MSFE. W tym momencie bardziej chodzi o to, komu imponujesz i kogo znasz, co doprowadzi Cię tam, gdzie chcesz być. Jeśli chcesz robić po stronie sprzedaży, front office lub M&A, zdobądź MBA.

Miary takie jak PKB, bezrobocie i płace mają na celu pomiar kondycji gospodarki. To są właściwe środki, na które warto zwrócić uwagę. Kiedy gospodarka ma się dobrze, ceny akcji zwykle też mają się dobrze, ale nie zawsze. Giełda wzrosła w 2009 roku o 23%, a gospodarka była w rozsypce. I nie powinno mieć znaczenia, która gazeta opublikowała te liczby. Jeśli liczby są prawidłowe, liczby są prawidłowe. Nie musisz słyszeć danych o PKB w wiadomościach z lisów, aby w nie uwierzyć.

">Tak, podatki płacą za infrastrukturę, ale nie są to „"cały powód, dla którego płacimy podatki w pierwszej kolejności."" Czy nawet nie czytałeś mojego posta? Powiedziałem już, że podatki płacą za infrastrukturę. Nie są one jednak "" cały powód, dla którego płacimy podatki w pierwszej kolejności”. „Są one częścią wielu rzeczy, za które płacą podatki. Jezus Chrystus”.

„Wcześniejsze skorzystanie z opcji, jeśli nie możesz sprzedać zakupionych akcji bazowych, jest generalnie niewskazane. Zasadniczo blokujesz najgorszą cenę, jaką możesz zapłacić, a ponadto tracisz wartość czasową swoich pieniędzy (co oznacza, że wprawdzie teraz dość niski, ale nadal). Załóżmy, że masz cenę wykonania 50 USD. Rozumiem, że uważasz, że akcje są warte więcej niż 50 USD. Załóżmy, że to prawdopodobnie, ale nie na pewno słuszne. Niezależnie od tego, czy są warte 51 USD, 151 USD lub 5100 USD, gdy Twoje opcje wygasną, nadal otrzymujesz zysk w wysokości 1 USD, 101 USD lub 5050 USD, jeśli poczekasz do wygaśnięcia i wykonasz wtedy. Wykonując teraz, rezygnujesz z dwóch rzeczy: Odsetki od pieniędzy płacisz za wykonanie od teraz do wygaśnięcia. Gwarancja, że ​​nic nie stracisz. Jeśli kupisz teraz, zyskujesz wszystkie korzyści powyżej ostrzeżenia, ale wszystkie wady są poniżej. Jeśli kupisz je później (w momencie wygaśnięcia ), nadal masz wszystkie plusy powyżej swojego strajku, ale nie ma minusów - w (as wydaje się mało prawdopodobne) w przypadku, gdy jest wart mniej niż strajk, nie możesz po prostu nic nie robić, zamiast mieć coś, co kupiłeś na strajku, co jest teraz mniej warte i ponieść tę stratę. Wykonując wcześnie, podejmujesz ryzyko straty i rezygnujesz z odsetek (lub „przenosisz”” pieniądze, które wydajesz na ćwiczenie) bez dodatkowych zwyżek. Możliwe, że istnieją ulgi podatkowe, jak wspominają inne plakaty, ale szanse na to, że „"start zegara"" dla LTCG są warte tyle, co ""opcjonalność"" lub ochrona przed stratami plus ""carry"", lub zainteresowanie, z którego rezygnujesz, jest mało prawdopodobne”.

Brzmi to oczywiste, ale: Jeśli właściciel jest niefrasobliwy, możesz go zapytać, czy nie ma nic przeciwko temu, że podnajmujesz przestrzeń, a następnie możesz ją podnająć. Oczywiście, być może będziesz musiał sam popracować, aby znaleźć odpowiedniego lokatora i upewnić się, że robisz wszystko zgodnie z prawem, ale jeśli to działa, to lepsze niż płacenie czynszu za darmo.

Zdziwiłbyś się, jak słaba jest koordynacja w niektórych dużych firmach. Pamiętam, jak sklep Walmart, w którym pracowałem, wysłał firmę, aby zainstalowała te nowe elektryczne suszarki do rąk w toaletach. Następnie, dwa tygodnie później, rozpoczęli kompletną przebudowę toalet, wyrywając suszarki do rąk, które właśnie zostały zainstalowane. Najwyraźniej ktoś wiedział, że zostaną zainstalowane suszarki do rąk, a ktoś wiedział, że toalety będą remontowane. Ale wyobrażam sobie, że obie strony nigdy ze sobą nie rozmawiały.


In [41]:
manual_annotations=[
{'Nanex', 'SEC'},
{'ETF', 'Trump'},
{'Reddit', 'SOPA','PIPA','NDAA'},
{'FSA','USD'},
{'USD', 'APR'},
{'Zarząd Portu','PA', 'Statua Wolności'},
{'2017 r.'},
set(),
{'USD'}, 
{'USD'}, 
{'Wall Street'},
{'ACA', 'Affordable Care Act'},
set(),
{'Wielkia Brytania'},
{'MFE', 'NYU', 'UCB', 'Carnegie Melon', 'Wall St.', 'Chicago', 'MSF','MSFE', 'M&A', 'MBA'},
{'PKB', '2009 rok'},
{'Jezus Chrystus'},
{'USD', 'LTCG'},
set(),
{'Walmart'}]

## Spacy NER Baseline

In [17]:
spacy_recog=[]
for passage in sampled_corpus:
    spacy_recog.append(get_entities_no_ner(passage["text"]))

c:\Users\janek\Documents\PJN\venv\Lib\site-packages\spacy\pipeline\attributeruler.py:149: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  matches = self.matcher(doc, allow_missing=True, as_spans=False)


## NER using LLAMA 3.2:3B

Setup with ollama

#### Example Zero-shot NER

Prompt: \
*Jakie jednostki nazwane (named entities) znajduja sie w tym zdaniu: \
Wczoraj w Krakowie miało miejsce spotkanie prezydentów Polski i Stanów Zjednoczonych. \
Odpowiedz podaj w formie zbioru (set). \
Jednostki nazwane powinny byc w formach podstawowych (zastosuj lematyzacje).*

In [24]:
from ollama import chat
from ollama import ChatResponse

def get_llama_response(message: str) -> ChatResponse:
    # Call the Llama model
    response: ChatResponse = chat(model='llama3.2:3b', messages=[{
        'role': 'user',
        'content': message
    }])
    return response.message.content

print(get_llama_response("""Jakie jednostki nazwane (named entities) znajduja sie w tym zdaniu: Wczoraj w Krakowie miało miejsce spotkanie prezydentów Polski i Stanów Zjednoczonych. Odpowiedz podaj w formie zbioru (set). Jednostki nazwane powinny byc w formach podstawowych (zastosuj lematyzacje)."""))

Oto lista jednostek nazwanych (named entities) z tego zdania:

{Kraków, Polska, Stanów Zjednoczonych, prezydenci}


#### Example 'few-shot' NER 

Prompt: 
Jednostki nazwane (named entities) znajdujace sie w tym zdaniu: 
"Wczoraj w Krakowie miało miejsce spotkanie prezydentów Polski i Stanów Zjednoczonych." 
to {"Kraków", "Polska", "Stany Zjednoczone"} 

Jednostki nazwane (named entities) znajdujace sie w tym zdaniu: 
"W październiku tego roku zgłoszono dziewięć projektów, poddanych następnie ocenie przez Morski Komitet Techniczny." 
to {"październik", "Morski Komitet Techniczny"} 

Jakie jednostki nazwane (named entities) znajduja sie w tym zdaniu: 
"Chram wywodzi swoje początki z VI wieku, ale w obecnej formie istnieje od 1168 roku."
Odpowiedz podaj w formie zbioru (set). 
Jednostki nazwane powinny byc w formach podstawowych (zastosuj lematyzacje). 

In [25]:
print(get_llama_response("""Jednostki nazwane (named entities) znajdujace sie w tym zdaniu: Wczoraj w Krakowie miało miejsce spotkanie prezydentów Polski i Stanów Zjednoczonych. to {"Kraków", "Polska", "Stany Zjednoczone"}. Jednostki nazwane (named entities) znajdujace sie w tym zdaniu: W październiku tego roku zgłoszono dziewięć projektów, poddanych następnie ocenie przez Morski Komitet Techniczny.
to {"październik", "Morski Komitet Techniczny"}. Jakie jednostki nazwane (named entities) znajduja sie w tym zdaniu: Chram wywodzi swoje początki z VI wieku, ale w obecnej formie istnieje od 1168 roku.Odpowiedz podaj w formie zbioru (set). Jednostki nazwane powinny byc w formach podstawowych (zastosuj lematyzacje)."""))

Oto jednostki nazwane (named entities) z danego zdania: {VI wiek, 1168 roku}


### NER recog using LLM and comparison to manual and spacy

In [ ]:
import re
def parse_llm_output(output):
    pattern = r"\{([^}]+)\}"
    matches = re.findall(pattern, output)
    result = {tuple(match.split(', ')) for match in matches}
    if not matches:
        print("Error parsing output: ", output)
        return set()
    return result

def create_ner_prompt(input):
    return f"""Jednostki nazwane (named entities) znajdujace sie w tym tekscie: 
Wczoraj w Krakowie miało miejsce spotkanie prezydentów Polski i Stanów Zjednoczonych. 
to {{Kraków, Polska, Stany Zjednoczone}}. 

Jednostki nazwane (named entities) znajdujace sie w tym zdaniu: 
W październiku tego roku zgłoszono dziewięć projektów, poddanych następnie ocenie przez Morski Komitet Techniczny. 
to {{październik, Morski Komitet Techniczny}} 

Jakie jednostki nazwane (named entities) znajduja sie w tym tekscie: {input}.

Odpowiedz podaj w formie zbioru (set). 

Jednostki nazwane powinny byc w formach podstawowych (zastosuj lematyzacje).
"""

llm_recog = []
for doc in sampled_corpus:
    prompt=create_ner_prompt(doc["text"])
    output = get_llama_response(prompt)
    llm_recog.append(parse_llm_output(output))
    

Error parsing output:  Oto lista jednostek nazwanych z tekstu:

* październik
* Morski Komitet Techniczny
* FSA (Federal Savings Association)
* USD (Dolar Stany Zjednoczone)

Podanie nie wymaga dodania "nie" w odpowiedzi.
Error parsing output:  Oto zestaw jednostek nazwanych w formach podstawowych:

* Kraków
* Polska
* Stany Zjednoczone
* Październik
* Morski Komitet Techniczny
* USD
* Kredyt hipoteczny
* APR (ang. Annual Percentage Rate)
* Płatność


## Evaluation

In [32]:
!pip install scikit-learn

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ------- -------------------------------- 2.1/11.1 MB 16.8 MB/s eta 0:00:01
   ----------------------------------- ---- 10.0/11.1 MB 31.0 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 26.7 MB/s eta 0:00:00
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [46]:
from sklearn.metrics import precision_recall_fscore_support

def eval(true_sets, pred_sets):
    # Initialize variables to store the total counts
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    # Calculate the metrics for each pair of sets
    for true_set, pred_set in zip(true_sets, pred_sets):
        true_positives += len(true_set & pred_set)  # True positives: intersection of sets
        false_positives += len(pred_set - true_set)  # False positives: elements in pred_set but not in true_set
        false_negatives += len(true_set - pred_set)  # False negatives: elements in true_set but not in pred_set

    # Compute Precision, Recall, and F1 Score using the formulae
    # Avoid division by zero by checking if the denominator is not zero

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    # Print the results
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    
print("Spacy vs. Manual")
eval(true_sets=manual_annotations, pred_sets=spacy_recog)
print()
print("LLAMA vs. Manual")
eval(true_sets=manual_annotations, pred_sets=llm_recog)


Spacy vs. Manual
Precision: 0.0
Recall: 0.0
F1 Score: 0

LLAMA vs. Manual
Precision: 0.0
Recall: 0.0
F1 Score: 0


In [47]:
print(spacy_recog)

[set(), set(), {'Alo'}, set(), {'Wiela'}, set(), {'Moj'}, set(), {'Moga'}, set(), {'Jasny', 'Walla'}, {'Alo'}, set(), set(), {'Walla', 'Ameryka'}, {'Kieda'}, set(), set(), set(), {'Alo'}]


In [48]:
print(llm_recog)

[{('SEC', 'rynki', 'międzynarodowy rynek', 'sekundy', 'godziny', 'sekundy')}, {('Gdańsk',), ('rynki', 'północnokoreański ogniem'), ('ETF', 'Polska', 'Stany Zjednoczone', 'Kraków'), ('październik', 'Morski Komitet Techniczny')}, {('Reddit', 'SOPA/PIPA', 'NDAA')}, set(), set(), {('Statu Wolności', 'PA', 'Zarząd Portu', 'Most Tapanzee')}, {('pierwszy_dzień_szkoły_rachunkowości', 'rachunkowość', 'dług', 'kapitał_właśny', 'zobowiązania', 'odsetki', 'firma', 'inwestycja', 'bilans', 'aktywa', 'pasywa', 'warunki_ustalone', 'warunki_uzgodnione', 'dyrektor_generalny', 'dyrektor_finansowy', 'akcjonariusze')}, {('organizacja', 'lista', 'spamer', 'prawny powód')}, {('październik', 'Morski Komitet Techniczny', 'bank', 'Stany Zjednoczone')}, {('Mathematics', 'USD', 'Tax')}, {('Człowiek', 'Wall Street', 'Bankier')}, {('ACA', 'Medicare', 'Koszty', 'Opieka Zdrowotna', 'NEJM')}, {('Dziadek', 'Kraków', 'Polska', 'Stany Zjednoczone', 'październik', 'Morski Komitet Techniczny')}, {('Wielka Brytania', 'firma